In [0]:
'''storage_account_name = "sadeltalivetables"
storage_account_key = "waOLVfBYqn9Chedmt9FTwc92yJhkmf+uAY60NloFYuB9hlkxRw01Nq58ZzsFu8WjTiBn+TwMmLP7+AStDMLHVA=="
container_name = "dlt"
mount_point = "/mnt/input"

configs = {
    "fs.azure.account.key.{0}.blob.core.windows.net".format(storage_account_name): storage_account_key
}

dbutils.fs.mount(
    source="wasbs://{0}@{1}.blob.core.windows.net".format(container_name, storage_account_name),
    mount_point=mount_point,
    extra_configs=configs
)'''

Out[23]: True

In [0]:
from pyspark.sql.functions import col, count, isnull, when
file_path = "/mnt/input/data/hidden_data_37680_records.csv"

# Read the CSV file into a Spark DataFrame
spark_df = spark.read.csv(file_path, header=True, inferSchema=True)
# Replace empty strings with null values
spark_df = spark_df.fillna("", subset=spark_df.columns)

# Check for null values in the DataFrame
spark_df.select([count(when(isnull(col), col)).alias(col) for col in spark_df.columns]).show()

+---+----+-----+
|_c0|text|class|
+---+----+-----+
|  0|   0|    0|
+---+----+-----+



In [0]:
spark_df.show(5)

+--------------------+--------------------+--------------------+
|                 _c0|                text|               class|
+--------------------+--------------------+--------------------+
|                   0|To be with him.Hi...|             suicide|
|                   1|Teenagers of Redd...|                    |
|Potentially manda...| as already I bel...| as short a while...|
|The main reason f...| which is a mista...| there are things...|
|Are you happy? Wh...|         non-suicide|                    |
+--------------------+--------------------+--------------------+
only showing top 5 rows



In [0]:
spark_df = spark_df.select("text", "class")
spark_df.show()

+--------------------+--------------------+
|                text|               class|
+--------------------+--------------------+
|To be with him.Hi...|             suicide|
|Teenagers of Redd...|                    |
| as already I bel...| as short a while...|
| which is a mista...| there are things...|
|         non-suicide|                    |
|I asked someone w...|         non-suicide|
|I am about to sle...|         non-suicide|
|Happy birthday to...|         non-suicide|
|I want to die so ...|             suicide|
|Last rent, I'm do...|                    |
|                    |                    |
| I'm a fat ugly p...|                    |
| to get better bu...| can't risk to pa...|
|                    |                    |
|                    |                    |
|                    |                    |
| note for potenti...| no one other nee...|
|                    |                    |
|             suicide|                    |
|Well boys i may b...|         n

In [0]:
%python
spark_df.createOrReplaceTempView("mentalhealthraw")

In [0]:
%sql
SELECT count(*) FROM mentalhealthraw WHERE class = 'suicide'

count(1)
9115


In [0]:
# create a permanent table
spark_df.write.format("parquet").saveAsTable("mentalhealth")

In [0]:
%sql
SELECT count(*) FROM mentalhealth WHERE class = 'suicide'

count(1)
9115
